<a href="https://colab.research.google.com/github/alisterpage/CHEM3580-Jupyter-Notebooks/blob/main/homework%203/water_heat_capacity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Homework - Molar Heat Capacity of Water

Using the code below:

1. Run a 100,000-step MD simulation of bulk water using the TIP3P water model. Use a 18.0 angstrom box, and a temperature of 298 K. Leave thermostat/barostat coupling constants unchanged from their current values.
1. Import the data in the tip3p_data.csv file into MS Excel. (_You may want to rename the file 'tip3p_data.csv' after each simulation, otherwise it will be overwritten by the next MD simulation_)
1. Inspect the Potential Energy data to determine whether or not your MD simulation has equilibrated.
1. Calculate the average potential energy, $<U>$, and its associated standard deviation, over an appropriate section of the MD trajectory.
1. Repeat steps 1-4 for the following simulation temperatures: 318 K, 338 K and 358 K.
1. Plot $<U>$ as a function of the simulation temperature, and use Excel's LINEST feature to determine the molar heat capacity of water and its associated +/- error.
1. Based on your results, do you think that TIP3P can accurately predict the heat capacity of water? Explain your answer.

**Submit your homework as a single MS excel file. Use a separate spreadsheet for each set of simulation results, and place your answer to steps (6) and (7) on a separate spreadsheet 'summary'. Ensure that your analysis, plots and answers are appropriately labelled and presented clearly.**
    

In [ ]:
#@title Setup Environment
!pip install -q condacolab
import condacolab
condacolab.install()
!conda install -c conda-forge packmol openmm -q 2>&1 >/dev/null
from openmm.app import *
from openmm import *
from openmm.unit import *
from sys import stdout
import re
import time
import numpy as np

In [ ]:
#@title Set Simulation Parameters
import ipywidgets as widgets
from IPython.display import display

#SIMULATION TEMPERATURE
# create a float text widget to input variable
temp_text = widgets.FloatText(
    value=298.0,  # default value
    description='Simulation Temperature (K):',
    step=1.0,  # step size
)
# adjust the layout of the widget
temp_text.layout.width = 'auto'
temp_text.style.description_width = 'initial'
# define a function to update the variable
def update_temp_text(change):
    global simTemperature
    simTemperature = change.new
# register the update function with the widget
temp_text.observe(update_temp_text, 'value')
# display the widget
display(temp_text)
# access the selected temperature value
simTemperature = temp_text.value*kelvin

#PRESSURE
# create a float text widget to input variable
temp_text = widgets.FloatText(
    value=1.0,  # default value
    description='Simulation pressure (atm):',
    step=0.10,  # step size
)
# adjust the layout of the widget
temp_text.layout.width = 'auto'
temp_text.style.description_width = 'initial'
# define a function to update the variable
def update_temp_text(change):
    global simPressure
    simPressure = change.new
# register the update function with the widget
temp_text.observe(update_temp_text, 'value')
# display the widget
display(temp_text)
# access the selected temperature value
simPressure = temp_text.value*atmospheres


#BOXSIZE
# create a float text widget to input variable
temp_text = widgets.FloatText(
    value=24.0,  # default value
    description='Box Size (Angstrom):',
    step=1.0,  # step size
)
# adjust the layout of the widget
temp_text.layout.width = 'auto'
temp_text.style.description_width = 'initial'
# define a function to update the variable
def update_temp_text(change):
    global boxsize
    boxsize = change.new
# register the update function with the widget
temp_text.observe(update_temp_text, 'value')
# display the widget
display(temp_text)
# access the selected temperature value
boxsize = temp_text.value

#THERMOSTAT TAU VALUE
# create a float text widget to input variable
temp_text = widgets.FloatText(
    value=0.1,  # default value
    description='Thermostat Coupling constant (/ps):',
    step=0.1,  # step size
)
# adjust the layout of the widget
temp_text.layout.width = 'auto'
temp_text.style.description_width = 'initial'
# define a function to update the variable
def update_temp_text(change):
    global simtau
    simtau = change.new
# register the update function with the widget
temp_text.observe(update_temp_text, 'value')
# display the widget
display(temp_text)
# access the selected temperature value
simtau = temp_text.value

#BAROSTAT TAU VALUE
# create a float text widget to input variable
temp_text = widgets.FloatText(
    value=1,  # default value
    description='Barostat Coupling constant (/time steps):',
    step=1,  # step size
)
# adjust the layout of the widget
temp_text.layout.width = 'auto'
temp_text.style.description_width = 'initial'
# define a function to update the variable
def update_temp_text(change):
    global simbarofreq
    simbarofreq = change.new
# register the update function with the widget
temp_text.observe(update_temp_text, 'value')
# display the widget
display(temp_text)
# access the selected temperature value
simbarofreq = temp_text.value


#TIMESTEP
# create a float text widget to input variable
temp_text = widgets.FloatText(
    value=1.0,  # default value
    description='Time step (fs):',
    step=0.5,  # step size
)
# adjust the layout of the widget
temp_text.layout.width = 'auto'
temp_text.style.description_width = 'initial'
# define a function to update the variable
def update_temp_text(change):
    global simTimestep
    simTimestep = change.new
# register the update function with the widget
temp_text.observe(update_temp_text, 'value')
# display the widget
display(temp_text)
# access the selected temperature value
simTimestep = temp_text.value*femtoseconds

#STEPS
# create a float text widget to input variable
temp_text = widgets.FloatText(
    value=100000,  # default value
    description='# MD steps:',
    step=100,  # step size
)
# adjust the layout of the widget
temp_text.layout.width = 'auto'
temp_text.style.description_width = 'initial'
# define a function to update the variable
def update_temp_text(change):
    global simNumSteps
    simNumSteps = change.new
# register the update function with the widget
temp_text.observe(update_temp_text, 'value')
# display the widget
display(temp_text)
# access the selected temperature value
simNumSteps = temp_text.value

In [ ]:
#@title Parameter Summary
density=0.99705
water=int(boxsize**3*density*1e6*(1e-10)**3/(18.01)*6.02214e23)
print("Box size of",boxsize,"angstroms at density,",density,"g/cm3 requires ",water," water molecules")
print(f"Simulation size: {water} waters")
print(f"Box size: {boxsize} Ang.")
print(f"Density: {density} g/cm3.")
print(f"Simulation temperature: {simTemperature} coupled every {simtau} ps")
print(f"Simulation pressure: {simPressure} coupled every {simbarofreq} MD time steps")
print(f"Simulation time step: {simTimestep}")
print(f"# MD steps: {simNumSteps} ; Total simulation length: {simNumSteps * simTimestep} = {simNumSteps*simTimestep / 1000 / 1000 / femtoseconds * nanoseconds}.")
pdbFile = 'system.pdb'

In [ ]:

#@title Build Simulation Box
with open("packmol.in", "w") as f:
    f.write("tolerance 2.0\n")
    f.write("filetype pdb\n")
    f.write("output system.pdb\n")
    f.write("\n")
    f.write("structure water.pdb\n")
    f.write("  number {0}\n".format(int(water)))
    f.write("  inside cube 2. 2. 2. {0}\n".format(boxsize))
    f.write("end structure\n")
    f.write("\n")
    f.write("add_box_sides 1.0\n")

text = '''ATOM      1  O   HOH A   1       4.125  13.679  13.761  1.00  0.00           O
ATOM      2  H1  HOH A   1       4.025  14.428  14.348  1.00  0.00           H
ATOM      3  H2  HOH A   1       4.670  13.062  14.249  1.00  0.00           H'''

with open('water.pdb', 'w') as f:
    f.write(text)

#%%bash
!packmol < packmol.in > packmol.out
with open('packmol.out', 'r') as f:
    text = f.read()
    print(text)
#with open('system.pdb', 'r') as f:
#    text = f.read()
#    print(text)

In [ ]:
#@title Setup MD Simulation

pdb = PDBFile('system.pdb')

tip3p_forcefield = ForceField('tip3p.xml')

tip3p_system = tip3p_forcefield.createSystem(
    pdb.topology,
    nonbondedMethod=PME,
    nonbondedCutoff=8*angstrom,
    constraints=HBonds,
    rigidWater=True)

tip3p_system.addForce(MonteCarloBarostat(simPressure, simTemperature, simbarofreq))

tip3p_integrator = NoseHooverIntegrator(
    simTemperature,
    simtau/picosecond,
    simTimestep
)
tip3p_simulation = Simulation(pdb.topology, tip3p_system, tip3p_integrator)
tip3p_simulation.context.setPositions(pdb.positions)

tip3p_simulation.reporters.append(DCDReporter('tip3p_data.dcd', 10))
tip3p_simulation.reporters.append(StateDataReporter(
    'tip3p_data.csv',
    10,
    step=True,
    temperature=True,
    potentialEnergy=True,
    kineticEnergy=True,
    totalEnergy=True,
    volume=True,
    density=True
))

In [ ]:
#@title Run Energy Minimisation
print("Running Energy Minimisation:")
t0 = time.time()
tip3p_simulation.minimizeEnergy()
t1 = time.time()
minTime = t1-t0
print("Minimisation took {} seconds".format(minTime))

In [ ]:
#@title Run MD Simulation

print('Start Simulation')
t0 = time.time()
tip3p_simulation.step(simNumSteps)
t1 = time.time()
simTime = t1-t0
print("Simulation took {} seconds for {} timesteps".format(simTime,simNumSteps))